# Advent of Code 2023

## Day 6

In [303]:
import aoc
aoc.read_input("cookie.txt", "input_2023", 2023, 6)

### Puzzle 1

Reflection: The trick to this one is to not loop through every possible combination, but to realize it is a math problem. The distance (d) the boat travels is velocity (v) times move time (m). The move time is equal to the race time (t) - button time (b). The velocity is equal to the button time. That means

d = vm

m = t - b

v = b

d = b(t - b)

d = -b<sup>2</sup> + tb

we need to now when distance is greater than the record distance (r), which is when

-b<sup>2</sup> + tb - r > 0

you can use the quadradic formula to figure out the values of b that satisfy this inequality

In [358]:
# parse the input
from collections import namedtuple
import math
from functools import reduce
Race = namedtuple("Race", "time distance")
with open("input_2023/6.txt") as f:
    racetimes = next(f).split(":")[1].split()
    racedistances = next(f).split(":")[1].split()
    races = [Race(int(time), int(dist)) for time, dist in zip(racetimes, racedistances)]

def num_win_options(a, b, c):
    # quadradic formula
    discriminant = math.sqrt(b**2 - 4*a*c)
    firstroot = (-1*b + discriminant)/(2*a)
    secondroot = (-1*b - discriminant)/(2*a)
    
    # find first integer larger than firstroot (min button time to win)
    min_button_time = firstroot+1 if firstroot.is_integer() else math.ceil(firstroot)
    
    # find last integer smaller than secondroot (max button time to win)
    max_button_time = secondroot-1 if secondroot.is_integer() else math.floor(secondroot)

    win_options = max_button_time - min_button_time + 1
    return win_options


num_ways_to_win = []

for race in races:
    ways_to_win = num_win_options(-1, race.time, -1*race.distance)
    num_ways_to_win.append(ways_to_win)

reduce((lambda x, y: x*y), num_ways_to_win)

2344708

### Puzzle 2

Reflection: You can solve the second puzzle the same way

In [360]:
# Parse the input
with open("input_2023/6.txt") as f:
    racetime = int(next(f).split(":")[1].replace(" ", ""))
    racedistance = int(next(f).split(":")[1].replace(" ", ""))

ways_to_win = num_win_options(-1, racetime, -1*racedistance)
print(ways_to_win)

30125202


## Day 5

In [1]:
import aoc
aoc.read_input("cookie.txt", "input_2023", 2023, 5)

### Puzzle 1

Reflection: Parsing the file was tricky, but my solution still didn't work even after I was sure I was parsing the file correctly. It turned out that the problem was after setting a new value from the mapping, it was using the same mapping (e.g. after changing soil to fertilizer, the code continued to use the soil-to-fertilizer map instead of moving to the fertilizer-to-water map). The skip_remaining value tells the loop to continue until it encounters a new mapping

In [208]:
# Parse puzzle input

def find_location(source, f):
    source_type = "seed"
    skip_remaining = False
    next(f) # skip first line w/ seed numbers
    for line in f:
        if line.strip().endswith(":"): # new map

            active_map = line.strip()[:-4]
            #print(f"{source_type}: {source}")
            source_type = active_map.split("-")[-1]
            skip_remaining = False
            continue
        
        if skip_remaining: # don't parse input until you get to a line defining a new mapping
            continue
        elif len(line.strip()) == 0: # empty lines
            continue
        else:
            dest_start, source_start, length  = [int(val) for val in line.split()]
            
            if source >= source_start and source <= source_start + length:
                source = dest_start + source - source_start
                skip_remaining = True
    return(source)
        
# Get seed numbers
with open("input_2023/5.txt") as f:
    seeds = [int(seed) for seed in next(f).split(":")[1].split()]

locs = []
for seed in seeds:
    with open("input_2023/5.txt") as f:
        loc = find_location(seed, f)
        locs.append(loc)
min(locs)

289863851

### Puzzle 2

Puzzle 1 deals with 20 seeds. Puzzle 2 deals with over 1 billion seeds. Looping over the entire file for every seed is not going to work. I tried a few different strategies, like converting the maps into graphs, but I was not able to solve this problem.

## Day 4    

In [393]:
import aoc
aoc.read_input("cookie.txt", "input_2023", 2023, 4)

### Puzzle 1

In [408]:
total = []
with open("input_2023/4.txt") as f:
    for rowtext in [line.split(":")[1].split("|") for line in aoc.parse_lines(f)]:
        winners, numbers = rowtext
        winners = winners.split()
        numbers = numbers.split()
        matches = 0
        for number in numbers:
            if number in winners:
                matches += 1
        score = 2 ** (matches - 1) if matches > 0 else 0
        total.append(score)
sum(total)


21138

### Puzzle 2

Reflection: I was worried that the brute force approach used here of several nested loops would result in an unacceptably long run time. That turned out not the be the case, however.

In [436]:
# Create a dictionary of cards in the form {cardnum1: nummatches, cardnum2: nummatches}
carddict = {}
with open("input_2023/4.txt") as f:
    for i, rowtext in enumerate([line.split(":")[1].split("|") for line in aoc.parse_lines(f)]):
        winners, numbers = rowtext
        winners = winners.split()
        numbers = numbers.split()
        matches = 0
        for number in numbers:
            if number in winners:
                matches += 1
        carddict[i+1] = matches

# Create a dictionary holding the number of copies of each card
cardcopies = {k:1 for k in carddict.keys()}

# Update the cardcopies dictionary
for cardnum, matches in carddict.items():
    for i in range(cardcopies[cardnum]):
        for i in range(cardnum+1, cardnum+matches+1):
            cardcopies[i] += 1

# total number of cards
sum(cardcopies.values())


7185540

In [422]:
carddict.items()

dict_items([(1, 4), (2, 2), (3, 2), (4, 1), (5, 0), (6, 0)])

## Day 3

In [122]:
import aoc
aoc.read_input("cookie.txt", "input_2023", 2023, 3)

### Puzzle 1

Reflection: I really struggled with this one. This solution is undoubtedly far more complex than necessary, but I was unable to find a simpler one.

In [389]:
# parse input into 2-d numpy array and get list of symbols
import numpy as np
symbols = []
inputmatrix = []
with open("input_2023/3.txt") as f:
    for rowtext in [line for line in aoc.parse_lines(f)]:
        row = []
        for char in rowtext:
            row.append(char)
            if not char.isdigit() and char != ".":
                if char not in symbols:
                    symbols.append(char)
        inputmatrix.append(row)

schematic = np.array(inputmatrix)

# pad the schematic to make moving window easier
 
schematic = np.pad(schematic, 1, "constant", constant_values=".")



In [327]:
# recursive function to extend window search to find part number, and the end column to search for symbols

def partnum(schematic, i, jstart, jend):
    number_window = schematic[i, jstart:jend]
    if all(char.isdigit() for char in number_window):
        partnumber, jend = partnum(schematic, i, jstart, jend+1)
    else: 
        partnumber = int("".join([char for char in number_window if char.isdigit()]))
    return partnumber, jend

In [392]:
height, width = schematic.shape
partnumbers = []
for i in range(1, height-1):
    continues = 0
    for j in range(1, width-1):
        
        # continue to skip past later digits in the part number
        if continues > 0:
                continues -= 1
                continue
        char = schematic[i, j]
        if char.isdigit():
            
            partnumber, jend = partnum(schematic, i, j, j+1)
            continues = jend - j - 2
            symbol_window = schematic[i-1:i+2, j-1:jend]
            for symbol in symbols:
                if symbol in symbol_window:
                    partnumbers.append(partnumber)
sum(partnumbers)
                    
                


535235

## Puzzle 2

Reflection: In Puzzle 1, I found symbols next to part numbers, as opposed to part numbers next to symbols. So I wasn't able to use as much of the part 1 solution as I would have liked. The function to rewrite part numbers into the numpy array still proved useful.

One other challenge was that part numbers can repeat, a pattern that isn't present in the example. I originally tried using sets and set length to find gear symbols that had exactly two different numbers, but that didn't work because part numbers are allowed to repeat (including repeating adjacent to the same symbol). Because of the way the rewriter function works, in a single row, repeating numbers adjacent to each other must belong to the same indvidual part (not different copies of the same part number). I learned that there is an itertools function groupby that collapses consecutive values in a list, which is useful for this.

This solution incorporates the solution to both puzzles.

In [391]:
height, width = schematic.shape
schematic_copy = schematic.astype("<U16")

from functools import reduce
from itertools import groupby 

# rewrite the schematic so that every cell with a part number includes the entire part number (not just a single digit)
for i in range(1, height-1):
    continues = 0
    for j in range(1, width-1):
        
        # continue to skip past later digits in the part number
        if continues > 0:
                continues -= 1
                schematic_copy[i, j] = partnumber
                continue
        char = schematic[i, j]
        if char.isdigit():
            
            partnumber, jend = partnum(schematic, i, j, j+1)
            continues = jend - j - 2
            schematic_copy[i, j] = str(partnumber)

# Find all symbols in input, check adjacent cells for part numbers
partnumbers = []
powers = []
for i in range(1, height-1):
    for j in range(1, width-1):
        if schematic[i, j] in symbols:
            parts = []
            schematic_window = schematic_copy[i-1:i+2, j-1:j+2]
            for row in schematic_window:
                 rowparts = [int(i[0]) for i in groupby(row) if i[0].isdigit()] # remove adjacent part numbers, which must be the same part
                 parts.extend(rowparts)
            partnumbers.extend(parts)
            if len(parts) == 2 and schematic[i, j] == "*": # find parts next to gears
                power = reduce((lambda x, y: int(x)*int(y)), parts)
                powers.append(power)
print(sum(partnumbers))
print(sum(powers))

535235
79844424


## Day 2

In [1]:
import aoc
aoc.read_input("cookie.txt", "input_2023", 2023, 2)

### Puzzle 1

Reflection: By far the hardest part of this puzzle for me was figuring out how to parse the input into a data structure that I could use. Once that was done, both puzzles were fairly easy.

In [99]:
# parse the input into a dictionary of lists of dictionaries of the form:
# {gameid: [{"blue": n, "red": n, "green": n}]}
with open("input_2023/2.txt") as f:
    lines = [line for line in f.read().strip().split("\n")]
    gamedict = {}
    for line in lines:
        gameid, results = line.split(":")
        gameid = int(gameid.split()[-1])
        results = results.strip()
        results = results.split(";")
        resultlist = []
        for result in results:
            draws = [draw.split() for draw in result.split(",")]
            drawdict = {k: int(v) for v, k in draws}
            resultlist.append(drawdict)
        results = resultlist
        gamedict[gameid] = results



# find if a single game had no reported dice draws exceed the maximum, else return 0

def resultchecker(results, maxdict):
    for result in results:
        for key in ["red", "green", "blue"]:
            if not key in result or result[key] <= maxdict[key]:
                pass
            else: return 0
    return game

maxdict = {
    "red": 12, 
    "green": 13, 
    "blue": 14
}

# Add the game ids of those games together
possiblegames = []
for game, results in gamedict.items():
    possiblegames.append(resultchecker(results, maxdict))

sum(possiblegames)



2528

### Puzzle 2

Reflection: I didn't know how to multiply all the elements of a list together. The code for the reduce function I looked up online, and didn't really understand how it worked at first. When I looked up reduce, I learned that reduce works by taking the first two elements of an iterable and passing them to a function. The result of that function and the next element in the iterable are passed to the function again, until the iterable is exhausted.

So in this case, the lambda function multiplies the two arguments. Reduce takes the first two elements of the list and passes them to the lambda function. Then it passes that result to the lambda function along with the next element (the original 3rd element). It just keeps doing that until it runs out of elements and returns the single result.

In [100]:

from functools import reduce 

powers = []

for game, results in gamedict.items():
    # find the min number of each color die in the bag (i.e. the max number of dice of that color in a single draw)
    mindict = {
        "red": 0,
        "green": 0,
        "blue": 0
    }
    for result in results:
        for dice, num in result.items():
            if num > mindict[dice]:
                mindict[dice] = num

    # Multiply the min dice values to find the "power" of each game
    power = reduce((lambda x, y: x * y), mindict.values())


    powers.append(power)

# Add the powers of each game together
sum(powers)

67363

## Day 1

In [3]:
import aoc
aoc.read_input("cookie.txt", "input_2023", 2023, 1)

Reflection: not super happy with this solution, feel like there's got to be a better way to do this.

### Puzzle 1

In [19]:
with open("input_2023/1.txt") as f:
    numlist = []
    for line in f:
        nums = "".join([char for char in line if char.isdigit()])
        num = int(f"{nums[0]}{nums[-1]}")
        numlist.append(num)
sum(numlist)

55090

Reflection: The second puzzle was pretty tricky. The overlapping number strings (like "eightwo") made parsing it much more difficult. This probably would have ben more elegant with regex. But I'm not good enough with regex to actually figure out how to do it: "I had a problem, so I tried to solve it with regex. Now I have two problems"

### Puzzle 2

In [51]:
numdict = {
    "1": "1",
    "2": "2",
    "3": "3",
    "4": "4",
    "5": "5",
    "6": "6",
    "7": "7",
    "8": "8",
    "9": "9",
    "one": "1",
    "two": "2",
    "three": "3",
    "four": "4",
    "five": "5",
    "six": "6",
    "seven": "7",
    "eight": "8",
    "nine": "9"
}
numlist = []
with open("input_2023/1.txt") as f:
    lines = [line for line in f.read().strip().split("\n")]
    numlist = []
    for line in lines:
        firstkey = sorted([(line.find(numtext), numtext) for numtext in numdict.keys() if line.find(numtext) >= 0])[0][1]
        lastkey = sorted([(line.rfind(numtext), numtext) for numtext in numdict.keys() if line.find(numtext) >= 0])[-1][1]
        num = int(f"{numdict[firstkey]}{numdict[lastkey]}")
        numlist.append(num)
sum(numlist)


281